## K-Nearest Neighbors (KNN)

## Importar librerías

In [8]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import optuna
from sklearn.model_selection import cross_val_score

c:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargar dataset

In [2]:
# Ruta del archivo
ruta = r'C:/Users/Administrator/Documents/Proyecto seg clientes refuerzo/Proyecto_seg_clientes/Data/teleCust1000t_listo.csv'

# Cargar el dataset
df = pd.read_csv(ruta)

# Verificar el contenido del dataset
print(df.head())
print(df.info())

   region  tenure  age  marital  address  income  ed  employ  retire  gender  \
0       2      13   44        1        9    64.0   4       5       0       0   
1       3      11   33        1        7   136.0   5       5       0       0   
2       3      68   52        1       24   116.0   1      29       0       1   
3       2      33   33        0       12    33.0   2       0       0       1   
4       2      23   30        1        9    30.0   1       2       0       0   

   reside  custcat  
0       2        1  
1       6        4  
2       2        3  
3       1        1  
4       4        3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1000 non-null   int64  
 1   tenure   1000 non-null   int64  
 2   age      1000 non-null   int64  
 3   marital  1000 non-null   int64  
 4   address  1000 non-null   int64  
 5   income   1000 no

## Preparar las características (X) y la variable objetivo (y)
Seleccionamos las columnas correspondientes:

In [3]:
# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['custcat'])  # Elimina la columna de la variable objetivo
y = df['custcat']                # Variable objetivo

## Normalizar las características
KNN es sensible a las escalas de los datos, por lo que es importante normalizar las características. Usaremos StandardScaler.

In [4]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Entrenar el modelo KNN
Creamos una instancia del modelo KNeighborsClassifier con n_neighbors=5 y lo entrenamos.

In [5]:
# Crear el modelo
model = KNeighborsClassifier(n_neighbors=5)

# Entrenar el modelo
model.fit(X_train, y_train)

KNeighborsClassifier()

## Hacer predicciones
Usamos el modelo para predecir las etiquetas en el conjunto de prueba.

In [6]:
# Hacer predicciones
y_pred = model.predict(X_test)

## Evaluar el modelo
Calculamos la exactitud y generamos un informe de clasificación.

In [7]:
# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Informe de clasificación
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))


Exactitud del modelo: 0.34
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.38      0.50      0.43        60
           2       0.23      0.26      0.24        39
           3       0.35      0.33      0.34        55
           4       0.33      0.20      0.25        46

    accuracy                           0.34       200
   macro avg       0.32      0.32      0.32       200
weighted avg       0.33      0.34      0.33       200



## Optimizar el modelo con Optuna
La función objetivo es donde Optuna probará diferentes valores para el número de vecinos (n_neighbors) y elegirá el que mejor rendimiento proporcione. Usaremos validación cruzada para evaluar el rendimiento.

In [9]:
# Función objetivo para Optuna
def objective(trial):
    # Sugerir un valor para n_neighbors
    n_neighbors = trial.suggest_int('n_neighbors', 1, 50)  # Buscamos en el rango de 1 a 50 vecinos
    
    # Crear el modelo con el número sugerido de vecinos
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    
    # Validación cruzada para evaluar el modelo
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

# Crear un estudio y optimizar
study = optuna.create_study(direction='maximize')  # Maximizar la exactitud
study.optimize(objective, n_trials=50)  # Realizar 50 intentos de optimización

# Mejor conjunto de hiperparámetros encontrado
print("Mejor conjunto de hiperparámetros:", study.best_params)
print("Mejor puntuación de validación cruzada:", study.best_value)

[I 2024-11-26 14:40:49,172] A new study created in memory with name: no-name-bcfba50a-b2d7-415a-a4ce-139d0775ff81
[I 2024-11-26 14:40:49,287] Trial 0 finished with value: 0.3275 and parameters: {'n_neighbors': 5}. Best is trial 0 with value: 0.3275.
[I 2024-11-26 14:40:49,387] Trial 1 finished with value: 0.3625 and parameters: {'n_neighbors': 20}. Best is trial 1 with value: 0.3625.
[I 2024-11-26 14:40:49,492] Trial 2 finished with value: 0.36875 and parameters: {'n_neighbors': 35}. Best is trial 2 with value: 0.36875.
[I 2024-11-26 14:40:49,577] Trial 3 finished with value: 0.30374999999999996 and parameters: {'n_neighbors': 2}. Best is trial 2 with value: 0.36875.
[I 2024-11-26 14:40:49,682] Trial 4 finished with value: 0.3725 and parameters: {'n_neighbors': 30}. Best is trial 4 with value: 0.3725.
[I 2024-11-26 14:40:49,791] Trial 5 finished with value: 0.35875 and parameters: {'n_neighbors': 11}. Best is trial 4 with value: 0.3725.
[I 2024-11-26 14:40:50,006] Trial 6 finished with

Mejor conjunto de hiperparámetros: {'n_neighbors': 50}
Mejor puntuación de validación cruzada: 0.3875


## Entrenar el modelo con los mejores hiperparámetros
Una vez que Optuna haya encontrado el mejor valor para n_neighbors, usaremos ese valor para entrenar el modelo final y evaluarlo en el conjunto de prueba.

In [10]:
# Obtener los mejores parámetros
best_n_neighbors = study.best_params['n_neighbors']

# Crear el modelo con el mejor número de vecinos
best_model = KNeighborsClassifier(n_neighbors=best_n_neighbors)

# Entrenar el modelo
best_model.fit(X_train, y_train)

# Hacer predicciones
y_pred = best_model.predict(X_test)

# Evaluar el modelo
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud en el conjunto de prueba: {accuracy:.2f}')

# Informe de clasificación
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))


Exactitud en el conjunto de prueba: 0.36
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.41      0.53      0.46        60
           2       0.30      0.15      0.20        39
           3       0.42      0.49      0.45        55
           4       0.22      0.17      0.20        46

    accuracy                           0.36       200
   macro avg       0.34      0.34      0.33       200
weighted avg       0.35      0.36      0.35       200

